In [18]:
# Duda 1: La UV 13106_11 no existe. Encontré 13106_11_A y 13106_11_B. ¿Se unen? 
# Duda 2: En el notebook '12b-Areal-data-to-polygon' encontré que para 'santiago' no se usa santiago_aoi de la base de datos, sino:
    #gdf = gpd.read_file('../../../data/processed/santiago/zonaurbana_zonascensales.gpkg')
    # ¿Hay un nuevo polígono que corresponde a AMS? Anteriormente yo cargaba el aoi de la base de datos (FROM projects_research.santiago_aoi)
# Duda 3: En la tabla ejemplo hay algunos Usos que se miden en (%) y otros en (Ha). Supongo que es un error y todo es en (%). Solo para confirmar.
# Duda 4: En la tabla ejemplo hay una columna llamada 'Porcentaje viviendas', ¿A qué se refiere?

## Import libraries

In [55]:
import geopandas as gpd
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import sys
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

In [2]:
all_local_dirs = 'alex'

In [96]:
if all_local_dirs == 'alex':
    aois_dir = "../../../data/external/temporal_todocker/santiago/aois/"
    data_dir = "../../../data/external/temporal_todocker/santiago/proximidad/"
    ndvi_dir = "../../../data/external/temporal_todocker/santiago/ndvi_santiago/"
#elif all_local_dirs == 'edgar':
    #grl_dir = "../../../data/processed/00_pois_formated/aereal_data/"
    #count_pois_dir = grl_dir
    #ndvi_dir = grl_dir
else:
    print("WARNING: Fix all_local_dirs variable")

## Load areas of interest

### AMS

In [28]:
# Load from database
query = 'SELECT * FROM projects_research.santiago_aoi'
santiago_ams = aup.gdf_from_query(query)

# Set CRS
try:
    santiago_ams = santiago_ams.set_crs("EPSG:4326")
except:
    santiago_ams = santiago_ams.to_crs("EPSG:4326")

# Simplifying format
santiago_ams['name'] = 'ams'
santiago_ams = santiago_ams[['name','geometry']]

# Show
print(santiago_ams.shape)
santiago_ams.head(2)

(1, 2)


,name,geometry
0,ams,"MULTIPOLYGON (((-70.69165 -33.80508, -70.69167..."


### Comunas

In [30]:
# Load all
all_comunas = gpd.read_file(aois_dir+'santiago_comunas_zonaurbana.geojson')

# Filter for comunas of interest
comunas_list = ['La Florida','Las Condes','Providencia','Pedro Aguirre Cerda','Quilicura']
comunas = all_comunas.loc[all_comunas.Comuna.isin(comunas_list)].copy()

# Simplifying format
comunas = comunas[['Comuna','geometry']]
comunas = comunas.rename(columns={'Comuna':'name'})

# Show
print(comunas.shape)
comunas.head(2)

(5, 2)


,name,geometry
6,Providencia,"MULTIPOLYGON (((-70.58484 -33.43126, -70.58445..."
20,La Florida,"MULTIPOLYGON (((-70.50766 -33.50052, -70.50770..."


### Unidades vecinales

In [37]:
# Load all
all_unidades_vecinales = gpd.read_file(aois_dir+'santiago_unidadesvecinales_zonaurbana.geojson')

# Filter for uvs of interest
uvs_list = ['13101_3','13101_15','13101_18','13101_49','13123_14','13106_08','13106_11_A','13106_11_B','13106_17','13117_19']
unidades_vecinales = all_unidades_vecinales.loc[all_unidades_vecinales.COD_UNICO_.isin(uvs_list)].copy()

# Join together uv 13106_11_A and 13106_11_B (Consultar si es necesario)

# Simplifiying format
unidades_vecinales['name'] = unidades_vecinales['COD_UNICO_']
unidades_vecinales = unidades_vecinales[['name','geometry']]

# Show
print(unidades_vecinales.shape)
unidades_vecinales.head(2)
unidades_vecinales

(10, 2)


,name,geometry
29,13101_15,"MULTIPOLYGON (((-70.67395 -33.44337, -70.67395..."
32,13101_18,"MULTIPOLYGON (((-70.66009 -33.44736, -70.66006..."
44,13101_3,"MULTIPOLYGON (((-70.65649 -33.43082, -70.65647..."
65,13101_49,"MULTIPOLYGON (((-70.63616 -33.44578, -70.63596..."
181,13106_11_A,"MULTIPOLYGON (((-70.68050 -33.45127, -70.68041..."
184,13106_08,"MULTIPOLYGON (((-70.69082 -33.45375, -70.69097..."
187,13106_11_B,"MULTIPOLYGON (((-70.68053 -33.45419, -70.68041..."
193,13106_17,"MULTIPOLYGON (((-70.71163 -33.46206, -70.71152..."
389,13117_19,"MULTIPOLYGON (((-70.70877 -33.45257, -70.70856..."
551,13123_14,"MULTIPOLYGON (((-70.62864 -33.43480, -70.62861..."


### Alameda, alameda oriente & alameda poniente

In [39]:
# ALAMEDA
if all_local_dirs == 'alex':
    alameda = gpd.read_file(aois_dir+"buffer_800m_nueva_alameda_32719.gpkg")
elif all_local_dirs == 'edgar':
    alameda = gpd.read_file(aois_dir + "buffer 800m nueva alameda.shp")
alameda['name'] = 'alameda'
alameda = alameda[['name','geometry']]
alameda = alameda.to_crs("EPSG:4326")

# ALAMEDA PONIENTE
alameda_poniente = gpd.read_file(aois_dir + 'alameda_poniente.gpkg')
alameda_poniente['name'] = 'alameda_poniente'
alameda_poniente = alameda_poniente[['name','geometry']]
alameda_poniente = alameda_poniente.to_crs("EPSG:4326")

# ALAMEDA ORIENTE
alameda_oriente = gpd.read_file(aois_dir + 'alameda_oriente.gpkg')
alameda_oriente['name'] = 'alameda_oriente'
alameda_oriente = alameda_oriente[['name','geometry']]
alameda_oriente = alameda_oriente.to_crs("EPSG:4326")

### Join all aois

In [40]:
all_aoi_gdfs = [santiago_ams,comunas,alameda,alameda_poniente,alameda_oriente,unidades_vecinales]
aoi_all = pd.concat(all_aoi_gdfs,
                    ignore_index = True, axis = 0)
aoi_all

,name,geometry
0,ams,"MULTIPOLYGON (((-70.69165 -33.80508, -70.69167..."
1,Providencia,"MULTIPOLYGON (((-70.58484 -33.43126, -70.58445..."
2,La Florida,"MULTIPOLYGON (((-70.50766 -33.50052, -70.50770..."
3,Las Condes,"MULTIPOLYGON (((-70.47897 -33.36728, -70.47897..."
4,Pedro Aguirre Cerda,"MULTIPOLYGON (((-70.67123 -33.51069, -70.67179..."
5,Quilicura,"MULTIPOLYGON (((-70.75679 -33.38348, -70.75679..."
6,alameda,"MULTIPOLYGON (((-70.70775 -33.45031, -70.69297..."
7,alameda_poniente,"MULTIPOLYGON (((-70.67975 -33.44347, -70.67894..."
8,alameda_oriente,"MULTIPOLYGON (((-70.67894 -33.45865, -70.67975..."
9,13101_15,"MULTIPOLYGON (((-70.67395 -33.44337, -70.67395..."


## Statistics for each area of interest's scorecard

In [41]:
# Población total <------------------- ¿Alguna capa en específico?
# Área (ha) ------------------------------------------> [Sale a partir de la geometría base]
# Densidad de población (pob/ha)
# [ALMOST DONE] Área total de construcción (m2) --------------------> usos_de_suelo_mzn_32719 [Suma de todas las áreas]
# [ALMOST DONE] Uso Habitacional(%) --------------------------------> usos_de_suelo_mzn_32719 (Solo la columna 'HABIT'?)
# [ALMOST DONE] Uso Equipamiento (%) -------------------------------> usos_de_suelo_mzn_32719 (¿¿¿¿????)
# [ALMOST DONE] Uso Comercio (%) -----------------------------------> usos_de_suelo_mzn_32719 (Sólo la columna 'COM'?)
# [ALMOST DONE] Uso Servicios (%) ----------------------------------> usos_de_suelo_mzn_32719 (¿¿¿¿????)
# [ALMOST DONE] Uso Industria (%) ----------------------------------> usos_de_suelo_mzn_32719 (Sólo la columna 'INDUS'?)
# [ALMOST DONE] Uso Otros (%) --------------------------------------> usos_de_suelo_mzn_32719 (Suma del resto de las columnas)
# [DONE] Total de viviendas ---------------------------------> viviendas_por_tipo_mzn_32719 [total_viv]
# Porcentaje viviendas ¿? <----------- ¿Qué es esto?
# [DONE] Viviendas sociales ----------------------------------> CECYT_CVS_2020_32719 [Total_de_v]
# [DONE] Casas(%) -------------------------------------------> viviendas_por_tipo_mzn_32719 [casas]
# [DONE] Departamentos (%) ----------------------------------> viviendas_por_tipo_mzn_32719 [deptos]
# Personas por vivienda <------------- ¿Alguna capa en específico?
# % Viviendas con hijos <------------- ¿Alguna capa en específico?
# [DONE] Total de oficinas ----------------------------------> shape_manzanas_oficinas_32719 [Oficinas]
# [ALMOST DONE] NDVI -----------------------------------------------> ¿Qué son los datos que se muestran en la tabla ejemplo? Al inicio pensé que se buscaba ndvi_mean.
# Nivel Socioeconómico promedio <----- ¿Alguna capa en específico?

In [90]:
data_gdf = aoi_all.copy()
data_gdf.set_index('name',inplace=True)
data_gdf

,geometry
name,
ams,"MULTIPOLYGON (((-70.69165 -33.80508, -70.69167..."
Providencia,"MULTIPOLYGON (((-70.58484 -33.43126, -70.58445..."
La Florida,"MULTIPOLYGON (((-70.50766 -33.50052, -70.50770..."
Las Condes,"MULTIPOLYGON (((-70.47897 -33.36728, -70.47897..."
Pedro Aguirre Cerda,"MULTIPOLYGON (((-70.67123 -33.51069, -70.67179..."
Quilicura,"MULTIPOLYGON (((-70.75679 -33.38348, -70.75679..."
alameda,"MULTIPOLYGON (((-70.70775 -33.45031, -70.69297..."
alameda_poniente,"MULTIPOLYGON (((-70.67975 -33.44347, -70.67894..."
alameda_oriente,"MULTIPOLYGON (((-70.67894 -33.45865, -70.67975..."


### __Total de viviendas, casas(%) y departamentos (%)__

In [91]:
# Load data
casas_deptos_mzn = gpd.read_file(data_dir + 'viviendas_por_tipo_mnz_32719.gpkg')

# Set CRS
try:
    casas_deptos_mzn = casas_deptos_mzn.set_crs("EPSG:4326")
except:
    casas_deptos_mzn = casas_deptos_mzn.to_crs("EPSG:4326")

# Show
print(casas_deptos_mzn.shape)
casas_deptos_mzn.head(2)

(48803, 18)


,objectid,cod_region,nom_region,cod_provin,nom_provin,cod_comuna,nom_comuna,nombre_dis,codigo_dis,manzent,zona_censa,manzana,total_viv,viv_ocupa,casas,deptos,otros,geometry
0,1,13,Metropolitana de Santiago,131,Santiago,13101,Santiago,MATADERO,21.0,1.310121e+13,2.0,11.0,100.0,83.0,4,89,7,"MULTIPOLYGON (((-70.64850 -33.47337, -70.64846..."
1,2,13,Metropolitana de Santiago,131,Santiago,13102,Cerrillos,DIVINO MAESTRO,3.0,1.310203e+13,5.0,35.0,20.0,20.0,20,0,0,"MULTIPOLYGON (((-70.73350 -33.49860, -70.73365..."


In [92]:
# Set current data of interest
data_of_interest = casas_deptos_mzn.copy()

# Find area of existing polygons (Used for representative data extraction)
data_of_interest = data_of_interest.to_crs("EPSG:32719")
data_of_interest['area'] = data_of_interest.area
data_of_interest = data_of_interest.to_crs("EPSG:4326")

# Data extraction by area of interest
for aoi_name in list(aoi_all.name.unique()):

    # Extract area of interest geometry
    aoi = aoi_all.loc[aoi_all.name==aoi_name].copy()

    # Clip data for area of interest
    data_of_interest_aoi = gpd.clip(data_of_interest, aoi)

    # Find percentage of geometry that fell inside clipped area
    data_of_interest_aoi = data_of_interest_aoi.to_crs("EPSG:32719")
    data_of_interest_aoi['clipped_area'] = data_of_interest_aoi.area
    data_of_interest_aoi = data_of_interest_aoi.to_crs("EPSG:4326")

    data_of_interest_aoi['pct_inside'] = data_of_interest_aoi['clipped_area']/data_of_interest_aoi['area']

    # -----------------------------------------------------------------------------------------
    # Register data
    # Total de viviendas
    data_of_interest_aoi['total_viv_inside'] = data_of_interest_aoi['total_viv'] * data_of_interest_aoi['pct_inside']
    data_gdf.loc[aoi_name,'Total de viviendas'] = data_of_interest_aoi['total_viv_inside'].sum()

    # % Casas
    data_of_interest_aoi['casas_inside'] = data_of_interest_aoi['casas'] * data_of_interest_aoi['pct_inside']
    data_gdf.loc[aoi_name,'Casas(%)'] = data_of_interest_aoi['casas_inside'].sum() / data_of_interest_aoi['total_viv_inside'].sum()

    # % Departamentos
    data_of_interest_aoi['deptos_inside'] = data_of_interest_aoi['deptos'] * data_of_interest_aoi['pct_inside']
    data_gdf.loc[aoi_name,'Departamentos(%)'] = data_of_interest_aoi['deptos_inside'].sum() / data_of_interest_aoi['total_viv_inside'].sum()

data_gdf

,geometry,Total de viviendas,Casas(%),Departamentos(%)
name,,,,
ams,"MULTIPOLYGON (((-70.69165 -33.80508, -70.69167...",2.204790e+06,0.653630,0.322322
Providencia,"MULTIPOLYGON (((-70.58484 -33.43126, -70.58445...",7.070061e+04,0.125324,0.870629
La Florida,"MULTIPOLYGON (((-70.50766 -33.50052, -70.50770...",1.201494e+05,0.757805,0.229023
Las Condes,"MULTIPOLYGON (((-70.47897 -33.36728, -70.47897...",1.177628e+05,0.308159,0.688778
Pedro Aguirre Cerda,"MULTIPOLYGON (((-70.67123 -33.51069, -70.67179...",2.985228e+04,0.833442,0.139822
Quilicura,"MULTIPOLYGON (((-70.75679 -33.38348, -70.75679...",6.220834e+04,0.768421,0.220131
alameda,"MULTIPOLYGON (((-70.70775 -33.45031, -70.69297...",1.237239e+05,0.151121,0.814889
alameda_poniente,"MULTIPOLYGON (((-70.67975 -33.44347, -70.67894...",3.204973e+04,0.367634,0.566149
alameda_oriente,"MULTIPOLYGON (((-70.67894 -33.45865, -70.67975...",9.167408e+04,0.075426,0.901850


### __Viviendas sociales__

In [81]:
# Load data
viviendas_sociales = gpd.read_file(data_dir + 'CECYT_CVS_2020_32719.gpkg')

# Set CRS
try:
    viviendas_sociales = viviendas_sociales.set_crs("EPSG:4326")
except:
    viviendas_sociales = viviendas_sociales.to_crs("EPSG:4326")

# Show
print(viviendas_sociales.shape)
viviendas_sociales.head(2)

(4412, 15)


,ID,REG,Comuna,Nombre_del,Programa,AGNO_PERMI,AGNO_RECEP,DIRECCION,Total_de_v,Asbesto_ce,Capa,CUT,Shape_STAr,Shape_STLe,geometry
0,02101CVS-001,02,02101,200 viv. Huamachuco,Fondo Solidario de Vivienda: Capitulo I (D.S. ...,2006,2009,Huamachuco - Pablo Neruda,200,No,Catastro de viviendas sociales,02101,25537.007812,756.225999,"MULTIPOLYGON (((-70.38324 -23.56441, -70.38263..."
1,02101CVS-002,02,02101,Puerta adentro,Fondo Solidario de Vivienda: Capitulo I (D.S. ...,2008,2009,Los Leones - San Francisco - Santa Lucia,11,No,Catastro de viviendas sociales,02101,835.877197,162.885405,"MULTIPOLYGON (((-70.37756 -23.60749, -70.37759..."


In [93]:
# Set current data of interest
data_of_interest = viviendas_sociales.copy()

# Find area of existing polygons (Used for representative data extraction)
data_of_interest = data_of_interest.to_crs("EPSG:32719")
data_of_interest['area'] = data_of_interest.area
data_of_interest = data_of_interest.to_crs("EPSG:4326")

# Data extraction by area of interest
for aoi_name in list(aoi_all.name.unique()):

    # Extract area of interest geometry
    aoi = aoi_all.loc[aoi_all.name==aoi_name].copy()

    # Clip data for area of interest
    data_of_interest_aoi = gpd.clip(data_of_interest, aoi)

    # Find percentage of geometry that fell inside clipped area
    data_of_interest_aoi = data_of_interest_aoi.to_crs("EPSG:32719")
    data_of_interest_aoi['clipped_area'] = data_of_interest_aoi.area
    data_of_interest_aoi = data_of_interest_aoi.to_crs("EPSG:4326")

    data_of_interest_aoi['pct_inside'] = data_of_interest_aoi['clipped_area']/data_of_interest_aoi['area']

    # -----------------------------------------------------------------------------------------
    # Register data
    # Viviendas sociales
    data_of_interest_aoi['Total_de_v_inside'] = data_of_interest_aoi['Total_de_v'] * data_of_interest_aoi['pct_inside']
    data_gdf.loc[aoi_name,'Viviendas sociales'] = data_of_interest_aoi['Total_de_v_inside'].sum()

data_gdf

,geometry,Total de viviendas,Casas(%),Departamentos(%),Viviendas sociales
name,,,,,
ams,"MULTIPOLYGON (((-70.69165 -33.80508, -70.69167...",2.204790e+06,0.653630,0.322322,281294.000000
Providencia,"MULTIPOLYGON (((-70.58484 -33.43126, -70.58445...",7.070061e+04,0.125324,0.870629,0.000000
La Florida,"MULTIPOLYGON (((-70.50766 -33.50052, -70.50770...",1.201494e+05,0.757805,0.229023,16351.362767
Las Condes,"MULTIPOLYGON (((-70.47897 -33.36728, -70.47897...",1.177628e+05,0.308159,0.688778,1732.000000
Pedro Aguirre Cerda,"MULTIPOLYGON (((-70.67123 -33.51069, -70.67179...",2.985228e+04,0.833442,0.139822,10169.009394
Quilicura,"MULTIPOLYGON (((-70.75679 -33.38348, -70.75679...",6.220834e+04,0.768421,0.220131,6225.000000
alameda,"MULTIPOLYGON (((-70.70775 -33.45031, -70.69297...",1.237239e+05,0.151121,0.814889,2591.483230
alameda_poniente,"MULTIPOLYGON (((-70.67975 -33.44347, -70.67894...",3.204973e+04,0.367634,0.566149,2591.483254
alameda_oriente,"MULTIPOLYGON (((-70.67894 -33.45865, -70.67975...",9.167408e+04,0.075426,0.901850,0.000000


### __Oficinas__

In [94]:
# Load data
oficinas = gpd.read_file(data_dir + 'shape_manzanas_oficinas_32719.gpkg')

# Set CRS
try:
    oficinas = oficinas.set_crs("EPSG:4326")
except:
    oficinas = oficinas.to_crs("EPSG:4326")

# Show
print(oficinas.shape)
oficinas.head(2)

(50974, 8)


,CMN_MZ,COMUNA,MANZ_SII,NOM_COMUNA,AREA,Sector,Oficinas,geometry
0,-99--99,-99,-99,QUILICURA,5.616492e+06,RM,0.0,"MULTIPOLYGON (((-70.71859 -33.38266, -70.71860..."
1,13101-0,13101,0,SANTIAGO CENTRO,4.306666e+05,RM,0.0,"MULTIPOLYGON (((-70.65904 -33.45278, -70.65887..."


In [95]:
# Set current data of interest
data_of_interest = oficinas.copy()

# Find area of existing polygons (Used for representative data extraction)
data_of_interest = data_of_interest.to_crs("EPSG:32719")
data_of_interest['area'] = data_of_interest.area
data_of_interest = data_of_interest.to_crs("EPSG:4326")

# Data extraction by area of interest
for aoi_name in list(aoi_all.name.unique()):

    # Extract area of interest geometry
    aoi = aoi_all.loc[aoi_all.name==aoi_name].copy()

    # Clip data for area of interest
    data_of_interest_aoi = gpd.clip(data_of_interest, aoi)

    # Find percentage of geometry that fell inside clipped area
    data_of_interest_aoi = data_of_interest_aoi.to_crs("EPSG:32719")
    data_of_interest_aoi['clipped_area'] = data_of_interest_aoi.area
    data_of_interest_aoi = data_of_interest_aoi.to_crs("EPSG:4326")

    data_of_interest_aoi['pct_inside'] = data_of_interest_aoi['clipped_area']/data_of_interest_aoi['area']

    # -----------------------------------------------------------------------------------------
    # Register data
    # Total de oficinas
    data_of_interest_aoi['Oficinas_inside'] = data_of_interest_aoi['Oficinas'] * data_of_interest_aoi['pct_inside']
    data_gdf.loc[aoi_name,'Total de oficinas'] = data_of_interest_aoi['Oficinas_inside'].sum()

data_gdf

,geometry,Total de viviendas,Casas(%),Departamentos(%),Viviendas sociales,Total de oficinas
name,,,,,,
ams,"MULTIPOLYGON (((-70.69165 -33.80508, -70.69167...",2.204790e+06,0.653630,0.322322,281294.000000,81287.500078
Providencia,"MULTIPOLYGON (((-70.58484 -33.43126, -70.58445...",7.070061e+04,0.125324,0.870629,0.000000,14559.196285
La Florida,"MULTIPOLYGON (((-70.50766 -33.50052, -70.50770...",1.201494e+05,0.757805,0.229023,16351.362767,760.175126
Las Condes,"MULTIPOLYGON (((-70.47897 -33.36728, -70.47897...",1.177628e+05,0.308159,0.688778,1732.000000,13190.785693
Pedro Aguirre Cerda,"MULTIPOLYGON (((-70.67123 -33.51069, -70.67179...",2.985228e+04,0.833442,0.139822,10169.009394,335.352338
Quilicura,"MULTIPOLYGON (((-70.75679 -33.38348, -70.75679...",6.220834e+04,0.768421,0.220131,6225.000000,1845.657306
alameda,"MULTIPOLYGON (((-70.70775 -33.45031, -70.69297...",1.237239e+05,0.151121,0.814889,2591.483230,17294.359083
alameda_poniente,"MULTIPOLYGON (((-70.67975 -33.44347, -70.67894...",3.204973e+04,0.367634,0.566149,2591.483254,450.365547
alameda_oriente,"MULTIPOLYGON (((-70.67894 -33.45865, -70.67975...",9.167408e+04,0.075426,0.901850,0.000000,16843.991759


### __NDVI__

In [99]:
# Load data
ndvi_gdf = gpd.read_file(ndvi_dir + f'Santiago_ndvi_HexRes10_v0.geojson')
ndvi_gdf = ndvi_gdf[['ndvi_mean','geometry']]

# Show
print(ndvi_gdf.shape)
ndvi_gdf.head(2)

(64190, 2)


,ndvi_mean,geometry
0,0.135445,"POLYGON ((-70.67723 -33.43729, -70.67786 -33.4..."
1,0.287743,"POLYGON ((-70.62472 -33.58049, -70.62535 -33.5..."


In [100]:
# Set current data of interest
data_of_interest = ndvi_gdf.copy()

# Data extraction by area of interest
for aoi_name in list(aoi_all.name.unique()):

    # Extract area of interest geometry
    aoi = aoi_all.loc[aoi_all.name==aoi_name].copy()

    # Clip data for area of interest
    data_of_interest_aoi = gpd.clip(data_of_interest, aoi)

    # -----------------------------------------------------------------------------------------
    # Register data
    # NDVI
    data_gdf.loc[aoi_name,'NDVI'] = data_of_interest_aoi['ndvi_mean'].mean()

data_gdf

,geometry,Total de viviendas,Casas(%),Departamentos(%),Viviendas sociales,Total de oficinas,NDVI
name,,,,,,,
ams,"MULTIPOLYGON (((-70.69165 -33.80508, -70.69167...",2.204790e+06,0.653630,0.322322,281294.000000,81287.500078,0.186410
Providencia,"MULTIPOLYGON (((-70.58484 -33.43126, -70.58445...",7.070061e+04,0.125324,0.870629,0.000000,14559.196285,0.218740
La Florida,"MULTIPOLYGON (((-70.50766 -33.50052, -70.50770...",1.201494e+05,0.757805,0.229023,16351.362767,760.175126,0.182567
Las Condes,"MULTIPOLYGON (((-70.47897 -33.36728, -70.47897...",1.177628e+05,0.308159,0.688778,1732.000000,13190.785693,0.268734
Pedro Aguirre Cerda,"MULTIPOLYGON (((-70.67123 -33.51069, -70.67179...",2.985228e+04,0.833442,0.139822,10169.009394,335.352338,0.114538
Quilicura,"MULTIPOLYGON (((-70.75679 -33.38348, -70.75679...",6.220834e+04,0.768421,0.220131,6225.000000,1845.657306,0.151646
alameda,"MULTIPOLYGON (((-70.70775 -33.45031, -70.69297...",1.237239e+05,0.151121,0.814889,2591.483230,17294.359083,0.125364
alameda_poniente,"MULTIPOLYGON (((-70.67975 -33.44347, -70.67894...",3.204973e+04,0.367634,0.566149,2591.483254,450.365547,0.132136
alameda_oriente,"MULTIPOLYGON (((-70.67894 -33.45865, -70.67975...",9.167408e+04,0.075426,0.901850,0.000000,16843.991759,0.119866


### __Usos, espacio construido__

In [102]:
# Load data
usos_construidos = gpd.read_file(data_dir + 'usos_de_suelo_mnz_32719.gpkg')

# Set CRS
try:
    usos_construidos = usos_construidos.set_crs("EPSG:4326")
except:
    usos_construidos = usos_construidos.to_crs("EPSG:4326")

# Show
print(usos_construidos.shape)
usos_construidos.head(2)

(50974, 26)


,CMN_MZ,COMUNA,MANZ_SI,NOM_COM,AREA,Sector,sprf_t_,ADMIN,AGR,BBCC,...,ESTAC,HABIT,HOTEL,INDUS,MIN,OFIC,OTROS,SALUD,TPTE,geometry
0,-99--99,-99,-99,QUILICURA,5.616492e+06,RM,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-70.71859 -33.38266, -70.71860..."
1,13101-0,13101,0,SANTIAGO CENTRO,4.306666e+05,RM,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-70.65904 -33.45278, -70.65887..."


In [104]:
# Set current data of interest
data_of_interest = usos_construidos.copy()

# Find area of existing polygons (Used for representative data extraction)
data_of_interest = data_of_interest.to_crs("EPSG:32719")
data_of_interest['area'] = data_of_interest.area
data_of_interest = data_of_interest.to_crs("EPSG:4326")

# Data extraction by area of interest
for aoi_name in list(aoi_all.name.unique()):

    # Extract area of interest geometry
    aoi = aoi_all.loc[aoi_all.name==aoi_name].copy()

    # Clip data for area of interest
    data_of_interest_aoi = gpd.clip(data_of_interest, aoi)

    # Find percentage of geometry that fell inside clipped area
    data_of_interest_aoi = data_of_interest_aoi.to_crs("EPSG:32719")
    data_of_interest_aoi['clipped_area'] = data_of_interest_aoi.area
    data_of_interest_aoi = data_of_interest_aoi.to_crs("EPSG:4326")

    data_of_interest_aoi['pct_inside'] = data_of_interest_aoi['clipped_area']/data_of_interest_aoi['area']

    # -----------------------------------------------------------------------------------------
    # Prepare data - All built uses
    all_uses = ['ADMIN','AGR','BBCC','BODEG','COM','CULT','DEPOR','EDUC','ERIAZO',
                'ESTAC','HABIT','HOTEL','INDUS','MIN','OFIC','OTROS','SALUD','TPTE']

    # Prepare data - clip built construction areas according to pct of block that falls inside aoi
    clipped_uses = []
    for use in all_uses:
        data_of_interest_aoi[f'{use}_inside'] = data_of_interest_aoi[use] * data_of_interest_aoi['pct_inside']
        clipped_uses.append(f'{use}_inside')

    # Find sum of all built area by block
    data_of_interest_aoi['tot_area'] = data_of_interest_aoi[clipped_uses].sum(axis=1)

    # -----------------------------------------------------------------------------------------
    # Register data - Uso Habitacional (%)
    area_habitacional = data_of_interest_aoi['HABIT_inside'].sum()
    data_gdf.loc[aoi_name,'Uso Habitacional (%)'] = area_habitacional / data_of_interest_aoi['tot_area'].sum()
    
    # Register data - Uso Equipamiento (%)
    area_equipamiento = (data_of_interest_aoi['CULT_inside'].sum() +
                         data_of_interest_aoi['DEPOR_inside'].sum() + 
                         data_of_interest_aoi['EDUC_inside'].sum() + 
                         data_of_interest_aoi['SALUD_inside'].sum())
    data_gdf.loc[aoi_name,'Uso Equipamiento (%)'] = area_equipamiento / data_of_interest_aoi['tot_area'].sum()
    
    # Register data - Uso Comercio (%)
    area_comercio = data_of_interest_aoi['COM_inside'].sum()
    data_gdf.loc[aoi_name,'Uso Comercio (%)'] = area_comercio / data_of_interest_aoi['tot_area'].sum()
    
    # Register data - Uso Servicios (%)
    area_servicios = (data_of_interest_aoi['ADMIN_inside'].sum() +
                      data_of_interest_aoi['HOTEL_inside'].sum() +
                      data_of_interest_aoi['OFIC_inside'].sum())
    data_gdf.loc[aoi_name,'Uso Servicios (%)'] = area_servicios / data_of_interest_aoi['tot_area'].sum()
    
    # Register data - Uso Industria (%)
    area_industria = (data_of_interest_aoi['BODEG_inside'].sum() +
                      data_of_interest_aoi['INDUS_inside'].sum())
    data_gdf.loc[aoi_name,'Uso Industria (%)'] = area_industria / data_of_interest_aoi['tot_area'].sum()
    
    # Register data - Uso Otros (%)
    area_otros = (data_of_interest_aoi['AGR_inside'].sum() +
                  data_of_interest_aoi['BBCC_inside'].sum() +
                  data_of_interest_aoi['ERIAZO_inside'].sum() +
                  data_of_interest_aoi['ESTAC_inside'].sum() +
                  data_of_interest_aoi['MIN_inside'].sum() +
                  data_of_interest_aoi['OTROS_inside'].sum() +
                  data_of_interest_aoi['TPTE_inside'].sum())
    data_gdf.loc[aoi_name,'Uso Otros (%)'] =  area_otros / data_of_interest_aoi['tot_area'].sum()

    # Register data - Área total de construcción (m2)
    data_gdf.loc[aoi_name,'Área total de construcción (m2)'] = area_habitacional + area_equipamiento + area_comercio + area_servicios + area_industria + area_otros

# Checking if all uses sum 100%
data_gdf['Uso_sum'] = (data_gdf['Uso Habitacional (%)']+data_gdf['Uso Equipamiento (%)']+data_gdf['Uso Comercio (%)']+data_gdf['Uso Servicios (%)']
+data_gdf['Uso Industria (%)']+data_gdf['Uso Otros (%)'])

# Show
data_gdf

,geometry,Total de viviendas,Casas(%),Departamentos(%),Viviendas sociales,Total de oficinas,NDVI,Uso Habitacional (%),Uso Equipamiento (%),Uso Comercio (%),Uso Servicios (%),Uso Industria (%),Uso Otros (%),Uso_sum,Área total de construcción (m2)
name,,,,,,,,,,,,,,,
ams,"MULTIPOLYGON (((-70.69165 -33.80508, -70.69167...",2.204790e+06,0.653630,0.322322,281294.000000,81287.500078,0.186410,0.598608,0.053593,0.064911,0.063563,0.119349,0.099975,1.0,2.460377e+08
Providencia,"MULTIPOLYGON (((-70.58484 -33.43126, -70.58445...",7.070061e+04,0.125324,0.870629,0.000000,14559.196285,0.218740,0.473017,0.066849,0.079129,0.147481,0.028894,0.204629,1.0,1.428125e+07
La Florida,"MULTIPOLYGON (((-70.50766 -33.50052, -70.50770...",1.201494e+05,0.757805,0.229023,16351.362767,760.175126,0.182567,0.757145,0.065767,0.074562,0.008689,0.015065,0.078772,1.0,1.055972e+07
Las Condes,"MULTIPOLYGON (((-70.47897 -33.36728, -70.47897...",1.177628e+05,0.308159,0.688778,1732.000000,13190.785693,0.268734,0.554998,0.028624,0.052549,0.102005,0.026063,0.235761,1.0,2.554819e+07
Pedro Aguirre Cerda,"MULTIPOLYGON (((-70.67123 -33.51069, -70.67179...",2.985228e+04,0.833442,0.139822,10169.009394,335.352338,0.114538,0.785554,0.058518,0.069738,0.014106,0.052730,0.019353,1.0,2.593497e+06
Quilicura,"MULTIPOLYGON (((-70.75679 -33.38348, -70.75679...",6.220834e+04,0.768421,0.220131,6225.000000,1845.657306,0.151646,0.308529,0.018828,0.026568,0.073362,0.530784,0.041930,1.0,6.660382e+06
alameda,"MULTIPOLYGON (((-70.70775 -33.45031, -70.69297...",1.237239e+05,0.151121,0.814889,2591.483230,17294.359083,0.125364,0.367504,0.088496,0.132240,0.209292,0.073720,0.128749,1.0,1.889179e+07
alameda_poniente,"MULTIPOLYGON (((-70.67975 -33.44347, -70.67894...",3.204973e+04,0.367634,0.566149,2591.483254,450.365547,0.132136,0.566846,0.091325,0.096893,0.033250,0.088214,0.123472,1.0,3.331470e+06
alameda_oriente,"MULTIPOLYGON (((-70.67894 -33.45865, -70.67975...",9.167408e+04,0.075426,0.901850,0.000000,16843.991759,0.119866,0.324825,0.087890,0.139808,0.246982,0.070617,0.129879,1.0,1.556032e+07
